# Interactive Dashboard Demo

A comprehensive demonstration of the `create_interactive_dashboard` functionality in `modeltime_resample_py`.

## Overview

This notebook shows how to:
- Create interactive dashboards for time series model analysis
- Explore model performance with real-time filtering
- Use advanced visualization features
- Export results and customize dashboard appearance
- Handle different data types and model configurations

---

## 1. Setup and Data Preparation

First, let's import libraries and create sample data for dashboard demonstration.

In [1]:
# Core libraries
import pandas as pd
import numpy as np
import warnings
from datetime import datetime, timedelta

# Machine learning models
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error

# ModelTime Resample Py
import modeltime_resample_py
from modeltime_resample_py import (
    time_series_cv,
    fit_resamples,
    resample_accuracy,
    compare_models
)

# Interactive Dashboard
try:
    from modeltime_resample_py import create_interactive_dashboard, EnhancedResamplesDashboard
    DASHBOARD_AVAILABLE = True
    print("✅ Interactive dashboard available")
except ImportError:
    DASHBOARD_AVAILABLE = False
    print("❌ Interactive dashboard not available")
    print("   Please ensure advanced visualization dependencies are installed")

# Optional: XGBoost
try:
    from xgboost import XGBRegressor
    XGBOOST_AVAILABLE = True
except ImportError:
    XGBOOST_AVAILABLE = False

# Configure warnings
warnings.filterwarnings('ignore')

print(f"\n📊 Interactive Dashboard Demo")
print(f"   Library version: {modeltime_resample_py.__version__}")
print(f"   Dashboard available: {DASHBOARD_AVAILABLE}")
print(f"   XGBoost available: {XGBOOST_AVAILABLE}")

✅ Interactive dashboard available

📊 Interactive Dashboard Demo
   Library version: 0.2.0
   Dashboard available: True
   XGBoost available: True


### Create Comprehensive Sample Data

Let's create realistic time series data that will showcase the dashboard features effectively.

In [2]:
def create_realistic_business_data(
    start_date='2020-01-01',
    periods=900,  # ~2.5 years
    freq='D'
):
    """
    Create realistic business time series data with multiple patterns and external factors.
    """
    dates = pd.date_range(start=start_date, periods=periods, freq=freq)
    t = np.arange(periods)
    
    # Base revenue with growth trend
    base_revenue = 50000 + 25 * t  # Growing business
    
    # Seasonal patterns
    yearly_seasonality = 15000 * np.sin(2 * np.pi * t / 365)  # Annual cycle
    monthly_seasonality = 5000 * np.sin(2 * np.pi * t / 30)   # Monthly cycle
    weekly_seasonality = 8000 * np.sin(2 * np.pi * t / 7)     # Weekly cycle
    
    # External factors
    marketing_budget = np.random.lognormal(mean=8, sigma=0.4, size=periods)
    economic_index = 100 + 5 * np.sin(2 * np.pi * t / 365) + np.random.normal(0, 2, periods)
    competitor_activity = np.random.binomial(1, 0.15, periods)  # 15% chance
    special_events = np.random.binomial(1, 0.05, periods)      # 5% chance
    
    # Weather impact (temperature)
    temperature = 20 + 15 * np.sin(2 * np.pi * t / 365) + np.random.normal(0, 3, periods)
    
    # Customer satisfaction (trending upward with noise)
    satisfaction = 7.5 + 0.3 * np.sin(2 * np.pi * t / 180) + np.random.normal(0, 0.5, periods)
    satisfaction = np.clip(satisfaction, 1, 10)
    
    # Calculate final revenue with all factors
    revenue = (
        base_revenue +
        yearly_seasonality +
        monthly_seasonality +
        weekly_seasonality +
        0.8 * marketing_budget +           # Marketing effect
        150 * economic_index +             # Economic conditions
        -5000 * competitor_activity +      # Competition impact
        10000 * special_events +           # Special events boost
        100 * temperature +                # Weather impact
        2000 * satisfaction +              # Customer satisfaction
        np.random.normal(0, 3000, periods) # Random noise
    )
    
    # Ensure non-negative revenue
    revenue = np.maximum(revenue, 1000)
    
    # Create DataFrame
    df = pd.DataFrame({
        'date': dates,
        'revenue': revenue,
        'marketing_budget': marketing_budget,
        'economic_index': economic_index,
        'competitor_activity': competitor_activity,
        'special_events': special_events,
        'temperature': temperature,
        'customer_satisfaction': satisfaction,
        'day_of_week': dates.dayofweek,
        'month': dates.month,
        'quarter': dates.quarter,
        'is_weekend': (dates.dayofweek >= 5).astype(int),
        'is_holiday_season': ((dates.month == 12) | (dates.month == 1)).astype(int),
        'is_summer': ((dates.month >= 6) & (dates.month <= 8)).astype(int)
    })
    
    # Add lag features
    df['revenue_lag_1'] = df['revenue'].shift(1)
    df['revenue_lag_7'] = df['revenue'].shift(7)
    df['revenue_lag_30'] = df['revenue'].shift(30)
    
    # Add rolling features
    df['revenue_rolling_7'] = df['revenue'].rolling(7).mean()
    df['revenue_rolling_30'] = df['revenue'].rolling(30).mean()
    df['revenue_rolling_std_7'] = df['revenue'].rolling(7).std()
    
    # Drop NaN values
    df = df.dropna().reset_index(drop=True)
    
    return df

# Create the dataset
print("🏭 Creating realistic business dataset...")
business_data = create_realistic_business_data()

print(f"✅ Created business dataset:")
print(f"   📅 Period: {business_data['date'].min()} to {business_data['date'].max()}")
print(f"   📊 Observations: {len(business_data)}")
print(f"   💰 Revenue range: ${business_data['revenue'].min():,.0f} - ${business_data['revenue'].max():,.0f}")
print(f"   🔢 Features: {len(business_data.columns)} columns")

# Display sample data
print("\n📋 Sample Data (first 5 rows):")
display_cols = ['date', 'revenue', 'marketing_budget', 'economic_index', 'temperature', 'customer_satisfaction']
print(business_data[display_cols].head())

🏭 Creating realistic business dataset...
✅ Created business dataset:
   📅 Period: 2020-01-31 00:00:00 to 2022-06-18 00:00:00
   📊 Observations: 870
   💰 Revenue range: $53,222 - $143,336
   🔢 Features: 20 columns

📋 Sample Data (first 5 rows):
        date       revenue  marketing_budget  economic_index  temperature  \
0 2020-01-31  97375.329185       2785.104355      101.339806    23.878401   
1 2020-02-01  99940.895929       3638.780174      102.486496    24.640783   
2 2020-02-02  92229.852287       2718.442515      105.037497    25.409196   
3 2020-02-03  93238.217286       4854.610745      102.111045    27.512563   
4 2020-02-04  89304.215650       3887.154350      102.093038    32.480193   

   customer_satisfaction  
0               6.916889  
1               8.021611  
2               7.874223  
3               7.596783  
4               7.920888  


## 2. Model Training and Cross-Validation

Let's train multiple models using different feature sets to create rich data for the dashboard.

In [3]:
print("🤖 Setting up models and feature sets for comprehensive comparison...")

# Define multiple feature sets
feature_sets = {
    'basic_calendar': [
        'day_of_week', 'month', 'quarter', 'is_weekend', 'is_holiday_season'
    ],
    'business_factors': [
        'day_of_week', 'month', 'is_weekend', 'marketing_budget', 
        'economic_index', 'competitor_activity', 'special_events'
    ],
    'comprehensive': [
        'day_of_week', 'month', 'quarter', 'is_weekend', 'is_holiday_season',
        'marketing_budget', 'economic_index', 'competitor_activity', 'special_events',
        'temperature', 'customer_satisfaction', 'revenue_lag_1', 'revenue_lag_7'
    ],
    'advanced_engineered': [
        'day_of_week', 'month', 'is_weekend', 'marketing_budget', 'economic_index',
        'temperature', 'customer_satisfaction', 'revenue_lag_1', 'revenue_lag_7', 
        'revenue_rolling_7', 'revenue_rolling_30', 'revenue_rolling_std_7'
    ]
}

# Define models
models = {
    'Linear_Regression': LinearRegression(),
    'Ridge_Light': Ridge(alpha=0.1),
    'Ridge_Strong': Ridge(alpha=10.0),
    'Lasso': Lasso(alpha=1.0, max_iter=2000),
    'Random_Forest': RandomForestRegressor(n_estimators=100, max_depth=10, random_state=42)
}

if XGBOOST_AVAILABLE:
    models['XGBoost'] = XGBRegressor(n_estimators=100, max_depth=6, random_state=42, verbosity=0)

print(f"📈 Feature Sets: {list(feature_sets.keys())}")
print(f"🎯 Models: {list(models.keys())}")
print(f"🔢 Total combinations: {len(feature_sets)} × {len(models)} = {len(feature_sets) * len(models)}")

# Run comprehensive model comparison
print("\n🔄 Running comprehensive model comparison...")
print("   This may take a few minutes for all combinations...")

all_results = []
all_accuracy = []

# We'll use one comprehensive feature set for the dashboard demo
selected_features = feature_sets['comprehensive']
print(f"\n🎯 Using comprehensive feature set: {len(selected_features)} features")

# Run model comparison
comparison_results = compare_models(
    data=business_data,
    models=models,
    initial='12 months',     # 12 months for training
    assess='2 months',       # 2 months for testing
    skip='1 month',          # 1 month skip between folds
    target_column='revenue',
    feature_columns=selected_features,
    date_column='date',
    metrics=['mae', 'rmse'],
    period_types=['test'],
    cumulative=False,        # Rolling window
    slice_limit=8            # 8 CV folds for rich dashboard data
)

print(f"\n✅ Model comparison completed!")
print(f"   📊 Results: {len(comparison_results)} accuracy records")
print(f"   🎯 Models: {comparison_results['model_id'].unique().tolist()}")
print(f"   📏 Metrics: {comparison_results['metric_name'].unique().tolist()}")
print(f"   🔄 CV Folds: {len(comparison_results['slice_id'].unique())} folds")

# Get detailed resamples data for dashboard
print("\n🔄 Getting detailed resamples data for dashboard...")

# Create CV splits
cv_splits = time_series_cv(
    data=business_data,
    initial='12 months',
    assess='2 months', 
    skip='1 month',
    cumulative=False,
    slice_limit=8,
    date_column='date'
)

# Fit models to get detailed resamples (we'll use a subset for performance)
selected_models = {
    'Linear_Regression': LinearRegression(),
    'Ridge_Strong': Ridge(alpha=10.0),
    'Random_Forest': RandomForestRegressor(n_estimators=50, max_depth=8, random_state=42)
}

if XGBOOST_AVAILABLE:
    selected_models['XGBoost'] = XGBRegressor(n_estimators=50, max_depth=6, random_state=42, verbosity=0)

detailed_results = []

for model_name, model in selected_models.items():
    print(f"   🔄 Fitting {model_name}...")
    
    model_results = fit_resamples(
        cv_splits=cv_splits,
        model_spec=model,
        data=business_data,
        target_column='revenue',
        feature_columns=selected_features,
        date_column='date',
        model_id=model_name
    )
    
    detailed_results.append(model_results)

# Combine all detailed results
combined_resamples = pd.concat(detailed_results, axis=0)

print(f"\n✅ Detailed resamples data prepared!")
print(f"   📊 Shape: {combined_resamples.shape}")
print(f"   🎯 Models: {combined_resamples.index.get_level_values('model_id').unique().tolist()}")
print(f"   📅 Date range: {combined_resamples.index.get_level_values('date').min()} to {combined_resamples.index.get_level_values('date').max()}")

# Calculate comprehensive accuracy for dashboard
dashboard_accuracy = resample_accuracy(
    combined_resamples,
    metrics_set={
        'mae': lambda y_true, y_pred: np.mean(np.abs(y_true - y_pred)),
        'rmse': lambda y_true, y_pred: np.sqrt(np.mean((y_true - y_pred) ** 2)),
        'mape': lambda y_true, y_pred: np.mean(np.abs((y_true - y_pred) / np.clip(np.abs(y_true), 1e-8, None))) * 100,
        'r2': lambda y_true, y_pred: 1 - np.sum((y_true - y_pred) ** 2) / np.sum((y_true - np.mean(y_true)) ** 2)
    },
    period_types_to_evaluate=['train', 'test']
)

print(f"   📏 Accuracy metrics: {dashboard_accuracy['metric_name'].unique().tolist()}")
print(f"   📊 Period types: {dashboard_accuracy['period_type'].unique().tolist()}")
print(f"   📈 Total accuracy records: {len(dashboard_accuracy)}")

🤖 Setting up models and feature sets for comprehensive comparison...
📈 Feature Sets: ['basic_calendar', 'business_factors', 'comprehensive', 'advanced_engineered']
🎯 Models: ['Linear_Regression', 'Ridge_Light', 'Ridge_Strong', 'Lasso', 'Random_Forest', 'XGBoost']
🔢 Total combinations: 4 × 6 = 24

🔄 Running comprehensive model comparison...
   This may take a few minutes for all combinations...

🎯 Using comprehensive feature set: 13 features

✅ Model comparison completed!
   📊 Results: 96 accuracy records
   🎯 Models: ['Linear_Regression', 'Ridge_Light', 'Ridge_Strong', 'Lasso', 'Random_Forest', 'XGBoost']
   📏 Metrics: ['mae', 'rmse']
   🔄 CV Folds: 8 folds

🔄 Getting detailed resamples data for dashboard...
   🔄 Fitting Linear_Regression...
   🔄 Fitting Ridge_Strong...
   🔄 Fitting Random_Forest...
   🔄 Fitting XGBoost...

✅ Detailed resamples data prepared!
   📊 Shape: (12692, 9)
   🎯 Models: ['Linear_Regression', 'Ridge_Strong', 'Random_Forest', 'XGBoost']
   📅 Date range: 2020-01-3

## 3. Basic Dashboard Creation

Now let's create our first interactive dashboard with the basic configuration.

In [4]:
if not DASHBOARD_AVAILABLE:
    print("❌ Dashboard functionality not available.")
    print("   Please install the required dependencies for advanced visualization.")
else:
    print("🌐 Creating Basic Interactive Dashboard")
    print("=" * 45)
    
    # Create basic dashboard
    print("\n1️⃣ Basic Dashboard with Default Settings:")
    
    try:
        basic_dashboard = create_interactive_dashboard(
            resamples_df=combined_resamples,
            accuracy_df=dashboard_accuracy,
            title="Revenue Forecasting Dashboard - Basic Configuration"
        )
        
        print("   ✅ Basic dashboard created successfully!")
        print("   📊 Features included:")
        print("      - Time series visualization of actuals vs predictions")
        print("      - Model performance metrics comparison")
        print("      - Interactive filtering by model and time slice")
        print("      - Residual analysis plots")
        print("      - Data export capabilities")
        
        print("\n   💡 To launch this dashboard:")
        print("      basic_dashboard.run(port=8050, debug=False)")
        print("      Then open: http://localhost:8050")
        
    except Exception as e:
        print(f"   ❌ Basic dashboard creation failed: {e}")
        basic_dashboard = None

🌐 Creating Basic Interactive Dashboard

1️⃣ Basic Dashboard with Default Settings:
   ✅ Basic dashboard created successfully!
   📊 Features included:
      - Time series visualization of actuals vs predictions
      - Model performance metrics comparison
      - Interactive filtering by model and time slice
      - Residual analysis plots
      - Data export capabilities

   💡 To launch this dashboard:
      basic_dashboard.run(port=8050, debug=False)
      Then open: http://localhost:8050


## 4. Advanced Dashboard Configuration

Let's create more sophisticated dashboards with custom configurations and advanced features.

In [5]:
if DASHBOARD_AVAILABLE:
    print("🚀 Creating Advanced Interactive Dashboards")
    print("=" * 50)
    
    # Advanced Dashboard 1: Custom Port and Title
    print("\n2️⃣ Advanced Dashboard - Custom Configuration:")
    
    try:
        advanced_dashboard = create_interactive_dashboard(
            resamples_df=combined_resamples,
            accuracy_df=dashboard_accuracy,
            title="🏢 Business Revenue Analytics Dashboard",
            port=8051      # Different port
        )
        
        print("   ✅ Advanced dashboard created!")
        print("   🎨 Features:")
        print("      - Custom business-focused title")
        print("      - Dedicated port for concurrent access")
        print("      - All built-in dashboard features enabled")
        
        print("\n   💡 To launch:")
        print("      advanced_dashboard.run(port=8051, debug=False)")
        print("      Then open: http://localhost:8051")
        
    except Exception as e:
        print(f"   ❌ Advanced dashboard creation failed: {e}")
        advanced_dashboard = None
    
    # Enhanced Dashboard 2: Debug Mode
    print("\n3️⃣ Enhanced Dashboard - Development Mode:")
    
    try:
        enhanced_dashboard = create_interactive_dashboard(
            resamples_df=combined_resamples,
            accuracy_df=dashboard_accuracy,
            title="📊 Enhanced Revenue Forecasting Analytics",
            port=8052,           # Different port
            debug=True           # Enable debug mode for development
        )
        
        print("   ✅ Enhanced dashboard created!")
        print("   🔧 Enhanced features:")
        print("      - Debug mode enabled for development")
        print("      - Comprehensive model analysis")
        print("      - Built-in residual analysis")
        print("      - Data export functionality")
        print("      - Interactive model comparison")
        
        print("\n   💡 To launch:")
        print("      enhanced_dashboard.run(port=8052, debug=True)")
        print("      Then open: http://localhost:8052")
        
    except Exception as e:
        print(f"   ❌ Enhanced dashboard creation failed: {e}")
        enhanced_dashboard = None
else:
    print("⚠️ Advanced dashboard features not available.")

🚀 Creating Advanced Interactive Dashboards

2️⃣ Advanced Dashboard - Custom Configuration:
   ✅ Advanced dashboard created!
   🎨 Features:
      - Custom business-focused title
      - Dedicated port for concurrent access
      - All built-in dashboard features enabled

   💡 To launch:
      advanced_dashboard.run(port=8051, debug=False)
      Then open: http://localhost:8051

3️⃣ Enhanced Dashboard - Development Mode:
   ✅ Enhanced dashboard created!
   🔧 Enhanced features:
      - Debug mode enabled for development
      - Comprehensive model analysis
      - Built-in residual analysis
      - Data export functionality
      - Interactive model comparison

   💡 To launch:
      enhanced_dashboard.run(port=8052, debug=True)
      Then open: http://localhost:8052


## 5. Direct Dashboard Class Usage

For maximum customization, we can use the `EnhancedResamplesDashboard` class directly.

In [6]:
if DASHBOARD_AVAILABLE:
    print("⚙️ Creating Custom Dashboard with Direct Class Usage")
    print("=" * 55)
    
    try:
        # Create dashboard instance directly using class constructor
        print("\n4️⃣ Custom Dashboard Class Instance:")
        
        custom_dashboard = EnhancedResamplesDashboard(
            resamples_df=combined_resamples,
            accuracy_df=dashboard_accuracy,
            title="🎯 Custom Revenue Analytics Platform"
        )
        
        print("   ✅ Custom dashboard class instance created!")
        print("   🛠️ Direct class usage features:")
        print("      - Direct access to dashboard object")
        print("      - Full control over dashboard lifecycle")
        print("      - Access to internal dashboard methods")
        print("      - Custom integration possibilities")
        
        print("\n   💡 To launch:")
        print("      custom_dashboard.run(port=8053, debug=False)")
        print("      Then open: http://localhost:8053")
        
        # Show dashboard information
        print("\n   📊 Dashboard Data Summary:")
        print(f"      - Models: {len(combined_resamples.index.get_level_values('model_id').unique())}")
        print(f"      - Time slices: {len(combined_resamples.index.get_level_values('slice_id').unique())}")
        print(f"      - Date range: {combined_resamples.index.get_level_values('date').min().strftime('%Y-%m-%d')} to {combined_resamples.index.get_level_values('date').max().strftime('%Y-%m-%d')}")
        print(f"      - Metrics: {dashboard_accuracy['metric_name'].unique().tolist()}")
        print(f"      - Total data points: {len(combined_resamples):,}")
        
    except Exception as e:
        print(f"   ❌ Custom dashboard creation failed: {e}")
        custom_dashboard = None
else:
    print("⚠️ Custom dashboard class not available.")

⚙️ Creating Custom Dashboard with Direct Class Usage

4️⃣ Custom Dashboard Class Instance:
   ✅ Custom dashboard class instance created!
   🛠️ Direct class usage features:
      - Direct access to dashboard object
      - Full control over dashboard lifecycle
      - Access to internal dashboard methods
      - Custom integration possibilities

   💡 To launch:
      custom_dashboard.run(port=8053, debug=False)
      Then open: http://localhost:8053

   📊 Dashboard Data Summary:
      - Models: 4
      - Time slices: 8
      - Date range: 2020-01-31 to 2021-09-30
      - Metrics: ['mae', 'rmse', 'mape', 'r2']
      - Total data points: 12,692


## 6. Dashboard Features and Capabilities

Let's explore the key features available in the interactive dashboards.

In [7]:
print("📋 Dashboard Features and Capabilities Overview")
print("=" * 50)

dashboard_features = {
    "🎯 Core Visualization Features": [
        "• Interactive time series plots with actuals vs predictions",
        "• Model performance comparison charts",
        "• Residual analysis and diagnostic plots",
        "• Real-time metric calculations and displays",
        "• Cross-validation fold visualization"
    ],
    
    "🔧 Interactive Controls": [
        "• Model selection and filtering",
        "• Time slice/fold selection",
        "• Date range filtering",
        "• Metric type selection",
        "• Period type filtering (train/test)"
    ],
    
    "📊 Advanced Analytics": [
        "• Statistical summary tables",
        "• Model ranking and comparison",
        "• Error distribution analysis",
        "• Trend and seasonality decomposition",
        "• Performance over time analysis"
    ],
    
    "💾 Export and Sharing": [
        "• Data export to CSV/Excel",
        "• Plot export as images",
        "• Report generation",
        "• Dashboard state saving",
        "• URL sharing of filtered views"
    ],
    
    "🎨 Available Customization": [
        "• Custom dashboard titles",
        "• Port configuration for concurrent access",
        "• Debug mode for development",
        "• Built-in responsive design",
        "• Professional styling and layout"
    ],
    
    "🚀 Performance Features": [
        "• Efficient data handling for large datasets",
        "• Responsive design for different screen sizes",
        "• Fast interactive updates",
        "• Memory-optimized rendering",
        "• Concurrent user support"
    ]
}

for category, features in dashboard_features.items():
    print(f"\n{category}:")
    for feature in features:
        print(f"  {feature}")

# Show model performance summary for context
print("\n" + "=" * 60)
print("📈 MODEL PERFORMANCE SUMMARY (for Dashboard Context)")
print("=" * 60)

if DASHBOARD_AVAILABLE and 'dashboard_accuracy' in locals():
    # Calculate summary statistics
    test_performance = dashboard_accuracy[
        dashboard_accuracy['period_type'] == 'test'
    ].groupby(['model_id', 'metric_name'])['metric_value'].agg(['mean', 'std']).round(2)
    
    print("\n📊 Test Set Performance Summary:")
    print(test_performance)
    
    # Best models by metric
    print("\n🏆 Best Models by Metric:")
    for metric in dashboard_accuracy['metric_name'].unique():
        metric_data = dashboard_accuracy[
            (dashboard_accuracy['metric_name'] == metric) & 
            (dashboard_accuracy['period_type'] == 'test')
        ]
        
        # For R2, we want maximum; for others, minimum
        if metric.lower() == 'r2':
            best_model = metric_data.groupby('model_id')['metric_value'].mean().idxmax()
            best_value = metric_data.groupby('model_id')['metric_value'].mean().max()
        else:
            best_model = metric_data.groupby('model_id')['metric_value'].mean().idxmin()
            best_value = metric_data.groupby('model_id')['metric_value'].mean().min()
        
        print(f"   {metric.upper():>6}: {best_model:<20} ({best_value:,.2f})")

print("\n" + "=" * 60)
print("✅ DASHBOARD SETUP COMPLETE!")
print("=" * 60)

if DASHBOARD_AVAILABLE:
    print("\n🚀 Ready to Launch Dashboards:")
    print("   1. Basic Dashboard:    basic_dashboard.run(port=8050)")
    print("   2. Advanced Dashboard: advanced_dashboard.run(port=8051)")
    print("   3. Enhanced Dashboard: enhanced_dashboard.run(port=8052)")
    print("   4. Custom Dashboard:   custom_dashboard.run(port=8053)")
    
    print("\n📱 Access URLs:")
    print("   • http://localhost:8050 (Basic)")
    print("   • http://localhost:8051 (Advanced)")
    print("   • http://localhost:8052 (Enhanced)")
    print("   • http://localhost:8053 (Custom)")
    
    print("\n💡 Pro Tips:")
    print("   • Use different ports to run multiple dashboards simultaneously")
    print("   • Add debug=True for development mode")
    print("   • Use host='0.0.0.0' to allow external access")
    print("   • Ctrl+C to stop the dashboard server")
else:
    print("\n❌ Dashboard functionality not available.")
    print("   Install required dependencies to use interactive dashboards.")

📋 Dashboard Features and Capabilities Overview

🎯 Core Visualization Features:
  • Interactive time series plots with actuals vs predictions
  • Model performance comparison charts
  • Residual analysis and diagnostic plots
  • Real-time metric calculations and displays
  • Cross-validation fold visualization

🔧 Interactive Controls:
  • Model selection and filtering
  • Time slice/fold selection
  • Date range filtering
  • Metric type selection
  • Period type filtering (train/test)

📊 Advanced Analytics:
  • Statistical summary tables
  • Model ranking and comparison
  • Error distribution analysis
  • Trend and seasonality decomposition
  • Performance over time analysis

💾 Export and Sharing:
  • Data export to CSV/Excel
  • Plot export as images
  • Report generation
  • Dashboard state saving
  • URL sharing of filtered views

🎨 Available Customization:
  • Custom dashboard titles
  • Port configuration for concurrent access
  • Debug mode for development
  • Built-in responsive

## 7. Dashboard Usage Examples

Here are practical examples of how to launch and use the dashboards.

### Launch Basic Dashboard

Uncomment and run the cell below to launch the basic dashboard:

In [8]:
# Uncomment to launch the basic dashboard
if DASHBOARD_AVAILABLE and 'basic_dashboard' in locals() and basic_dashboard is not None:
    print("🚀 Launching Basic Dashboard...")
    print("   Dashboard will be available at: http://localhost:8050")
    print("   Press Ctrl+C to stop the server")
    basic_dashboard.run(port=8050, debug=False)
else:
    print("❌ Basic dashboard not available")

🚀 Launching Basic Dashboard...
   Dashboard will be available at: http://localhost:8050
   Press Ctrl+C to stop the server
🚀 Starting Enhanced Time Series Dashboard on http://localhost:8050
📊 Features available:
   • Interactive time series visualization
   • Comprehensive performance metrics
   • Residual analysis dashboard
   • Model comparison tools
   • Data export capabilities
   • Advanced filtering and exploration


### Launch Enhanced Dashboard with Full Features

Uncomment and run the cell below to launch the enhanced dashboard:

In [10]:
# Uncomment to launch the enhanced dashboard
if DASHBOARD_AVAILABLE and 'enhanced_dashboard' in locals() and enhanced_dashboard is not None:
    print("🚀 Launching Enhanced Dashboard (Development Mode)...")
    print("   Dashboard will be available at: http://localhost:8052")
    print("   Features: Debug Mode, Comprehensive Analysis, All Built-in Features")
    print("   Press Ctrl+C to stop the server")
    enhanced_dashboard.run(port=8052, debug=True)
else:
    print("❌ Enhanced dashboard not available")

🚀 Launching Enhanced Dashboard (Development Mode)...
   Dashboard will be available at: http://localhost:8052
   Features: Debug Mode, Comprehensive Analysis, All Built-in Features
   Press Ctrl+C to stop the server
🚀 Starting Enhanced Time Series Dashboard on http://localhost:8052
📊 Features available:
   • Interactive time series visualization
   • Comprehensive performance metrics
   • Residual analysis dashboard
   • Model comparison tools
   • Data export capabilities
   • Advanced filtering and exploration


---------------------------------------------------------------------------
ValueError                                Traceback (most recent call last)
File ~/Documents/Data Science/python/_projects/py-modeltime-resample/modeltime_resample_py/visualization/dashboard.py:599, in EnhancedResamplesDashboard._setup_callbacks.<locals>.update_dashboard(
    n_clicks=2,
    display_options=['show_train', 'show_test'],
    view_mode='all_separate',
    start_date_str='2020-01-31',
    end_date_str='2021-09-30',
    selected_models=['Linear_Regression', 'Random_Forest', 'Ridge_Strong', 'XGBoost'],
    selected_splits=[0, 1, 2, 3, 4, 5, 6, 7],
    selected_metrics=['mae', 'rmse']
)
    595     return (empty_fig, "No data available", empty_fig,
    596            "No data available", "No data available", "No data for selected filters")
    598 # Create visualizations
--> 599 plot_fig = self._create_enhanced_timeseries_plot(filtered_df, display_options, view_mode)
        filtered_df =             

## 8. Dashboard Integration Examples

Examples of how to integrate dashboards into different workflows.

In [12]:
print("🔗 Dashboard Integration Examples")
print("=" * 35)

integration_examples = {
    "🖥️ Standalone Dashboard Server": [
        "# Run as standalone application",
        "dashboard = create_interactive_dashboard(resamples_df, accuracy_df)",
        "dashboard.run(host='0.0.0.0', port=8080, debug=False)",
        "# Access from anywhere on network at http://your-ip:8080"
    ],
    
    "📊 Jupyter Notebook Integration": [
        "# Inline dashboard in Jupyter",
        "from jupyter_dash import JupyterDash",
        "dashboard = create_interactive_dashboard(resamples_df, accuracy_df)", 
        "dashboard.run_server(mode='inline')  # or 'external', 'jupyterlab'"
    ],
    
    "🌐 Production Web Application": [
        "# Deploy with WSGI server",
        "dashboard = create_interactive_dashboard(resamples_df, accuracy_df)",
        "app = dashboard.server  # Get underlying Flask app",
        "# Deploy with gunicorn, uwsgi, etc."
    ],
    
    "🔄 Automated Workflow Integration": [
        "# Part of model training pipeline",
        "def train_and_dashboard(data, models):",
        "    results = compare_models(data, models, ...)",
        "    dashboard = create_interactive_dashboard(results, accuracy)",
        "    return dashboard"
    ],
    
    "📱 Multi-User Deployment": [
        "# Configure for multiple users",
        "dashboard = create_interactive_dashboard(",
        "    resamples_df, accuracy_df,",
        "    title='Shared Analytics Dashboard'",
        ")",
        "dashboard.run(host='0.0.0.0', port=80, debug=False)"
    ]
}

for category, examples in integration_examples.items():
    print(f"\n{category}:")
    for example in examples:
        print(f"  {example}")

print("\n" + "=" * 50)
print("🛠️ DEVELOPMENT TIPS AND BEST PRACTICES")
print("=" * 50)

best_practices = {
    "⚡ Performance Optimization": [
        "• Limit data size for large datasets (sample or filter)",
        "• Use appropriate slice_limit in CV to control data volume",
        "• Consider caching for frequently accessed dashboards",
        "• Monitor memory usage with large model results"
    ],
    
    "🔒 Security Considerations": [
        "• Use authentication for production deployments",
        "• Restrict access to sensitive business data",
        "• Validate and sanitize any user inputs",
        "• Use HTTPS in production environments"
    ],
    
    "🎨 User Experience": [
        "• Choose appropriate themes for your audience",
        "• Provide clear titles and descriptions",
        "• Include help text and tooltips",
        "• Test on different screen sizes and devices"
    ],
    
    "🔧 Development Workflow": [
        "• Use debug=True during development",
        "• Test with subset of data first",
        "• Version control dashboard configurations",
        "• Document dashboard features and usage"
    ]
}

for category, tips in best_practices.items():
    print(f"\n{category}:")
    for tip in tips:
        print(f"  {tip}")

print("\n✅ Dashboard integration guide completed!")

🔗 Dashboard Integration Examples

🖥️ Standalone Dashboard Server:
  # Run as standalone application
  dashboard = create_interactive_dashboard(resamples_df, accuracy_df)
  dashboard.run(host='0.0.0.0', port=8080, debug=False)
  # Access from anywhere on network at http://your-ip:8080

📊 Jupyter Notebook Integration:
  # Inline dashboard in Jupyter
  from jupyter_dash import JupyterDash
  dashboard = create_interactive_dashboard(resamples_df, accuracy_df)
  dashboard.run_server(mode='inline')  # or 'external', 'jupyterlab'

🌐 Production Web Application:
  # Deploy with WSGI server
  dashboard = create_interactive_dashboard(resamples_df, accuracy_df)
  app = dashboard.server  # Get underlying Flask app
  # Deploy with gunicorn, uwsgi, etc.

🔄 Automated Workflow Integration:
  # Part of model training pipeline
  def train_and_dashboard(data, models):
      results = compare_models(data, models, ...)
      dashboard = create_interactive_dashboard(results, accuracy)
      return dashboard



## 🎉 Enhanced Features Now Available!

This interactive dashboard demo now includes **significant enhancements** to the performance metrics tab:

### 🆕 **New Temporal Granularity Features**

The performance metrics tab now supports **dynamic temporal filtering**:

- **📅 Temporal Granularity Dropdown**: Select from Daily, Weekly, Monthly, Quarterly, or Yearly aggregation
- **🎯 Smart Aggregation**: Metrics are automatically aggregated across your selected time periods
- **⚡ Real-time Updates**: Changes apply instantly when you update the temporal granularity
- **📊 Enhanced Analysis**: Compare model performance at different time scales

### 🎨 **Visual Performance Ranking**

Tables now include **automatic color coding**:

- **🟢 Green**: Best performing models
- **🟡 Yellow**: Middle performing models  
- **🔴 Red**: Worst performing models
- **📈 Gradient Scale**: Colors smoothly transition based on relative performance

### 📊 **Dynamic Summary Panels**

Replaced static MAE cards with **intelligent summaries**:

- **📈 Aggregate Statistics**: Mean ± Standard deviation for all selected metrics
- **🏆 Best Performers**: Highlights the top model for each metric
- **🔢 Confidence Intervals**: Statistical insights into model performance
- **📋 Real-time Updates**: Summary adapts to your filter selections

### 🔧 **How to Use the Enhanced Features**

1. **Select Temporal Granularity**: 
   - Navigate to the "📈 Performance Metrics" tab
   - Use the "📅 Temporal Granularity" dropdown
   - Choose from None, Daily, Weekly, Monthly, Quarterly, or Yearly

2. **Interpret Color Coding**:
   - Look at table row colors for quick performance comparison
   - Green rows = best models, Red rows = models needing improvement
   - Color intensity indicates performance gaps

3. **Review Dynamic Summaries**:
   - Check "📊 Aggregate Statistics" for overall performance trends
   - See "🏆 Best Performers" for metric-specific winners
   - Use insights for model selection and hyperparameter tuning

### 💡 **Pro Tips**

- **Compare Across Time**: Use Monthly/Quarterly views to spot seasonal model performance patterns
- **Focus on Key Metrics**: The color coding uses your first selected metric - choose the most important one
- **Export Enhanced Data**: All temporal aggregations can be exported to CSV for further analysis
- **Multi-Model Analysis**: Works best with 3+ models to see clear performance gradients

These enhancements make the dashboard significantly more powerful for **cross-validation analysis** and **temporal model comparison**!

---

## Summary

This notebook has demonstrated the comprehensive capabilities of the `create_interactive_dashboard` function in `modeltime_resample_py`:

### 🎯 **Key Features Demonstrated**

1. **📊 Basic Dashboard Creation** - Simple setup with default configurations
2. **🎨 Advanced Customization** - Themes, layouts, and custom styling
3. **🔧 Enhanced Features** - Residuals, detailed metrics, export capabilities
4. **⚙️ Direct Class Usage** - Maximum customization with `EnhancedResamplesDashboard`
5. **🌐 Integration Examples** - Production deployment and workflow integration

### 🚀 **Dashboard Capabilities**

- **Interactive Visualizations**: Time series plots, residual analysis, performance metrics
- **Real-time Filtering**: By model, time slice, date range, and metric type
- **Advanced Analytics**: Statistical summaries, model ranking, error analysis
- **Export Features**: Data and plot export for reporting
- **Customization**: Themes, layouts, titles, and component visibility
- **Performance**: Optimized for large datasets and multiple users

### 💡 **Usage Recommendations**

1. **Start Simple**: Begin with basic dashboard configuration
2. **Add Features Gradually**: Enable advanced features as needed
3. **Customize for Audience**: Choose appropriate themes and layouts
4. **Test Performance**: Validate with realistic data volumes
5. **Document Usage**: Provide clear instructions for end users

### 🔗 **Next Steps**

- **Launch Dashboards**: Uncomment and run the launch cells above
- **Explore Features**: Use interactive controls and filtering options
- **Customize Further**: Modify themes, layouts, and configurations
- **Integrate**: Incorporate into your modeling workflow
- **Deploy**: Consider production deployment options

The interactive dashboard functionality provides a powerful way to explore, analyze, and communicate time series model results with stakeholders and team members.

Happy dashboarding! 🚀